
| 추천 방식 | 특징 | 장점 | 단점 |
|---|---|---|---|
| **임베딩 기반 추천** | 문맥적 의미 및 분위기 분석 | 사용자의 모호한 분위기 파악 가능 | 특정 키워드(예: 메뉴, 룸) 반영 어려움 |
| **키워드 기반 추천** | 특정 단어(메뉴, 서비스, 분위기)를 직접 매칭 | 사용자의 명확한 요구 반영 가능 | 분위기 같은 추상적 개념 반영 어려움 |

**➡ 키워드 매칭과 임베딩 기반 추천을 결합하는 하이브리드 방법을 도입!**

---

- **TF-IDF (Term Frequency-Inverse Document Frequency)**: 사용자 입력 문장과 음식점 데이터(메뉴, 리뷰, 설명)를 비교하여 키워드 중요도를 계산.
- **BM25 (Best Matching 25)**: TF-IDF보다 더 정교한 검색 모델로, 검색 엔진에서도 사용됨.


- **Instructor-xl, MPNet, MiniLM 등 임베딩 모델 활용**
- 문장의 전체적인 의미와 분위기를 반영하여 키워드 기반 추천의 한계를 보완


### 하이브리드 추천 시스템 (키워드 + 임베딩 결합)
1. **키워드 기반 (TF-IDF 또는 BM25)으로 상위 N개 식당 후보군을 선정**
2. **임베딩 기반 추천으로 최종 순위 결정**



## 기대 효과
**임베딩 모델의 한계를 보완** → 키워드를 활용하여 특정 요소(soju, BBQ 등)를 반영 가능  
**사용자의 명확한 요청 반영 가능** → "BBQ & Soju" 요청 시 BBQ와 Soju가 있는 식당을 먼저 추천  
**정확도 향상 및 유연한 조합 가능** → 분위기 및 키워드를 동시에 고려하여 최적의 추천 제공  

---



In [25]:
import numpy as np
from sentence_transformers import SentenceTransformer
from numpy import dot
from numpy.linalg import norm

# ===============================
# 1. 모델 로드
# ===============================
model_names = {
    "MiniLM": "sentence-transformers/all-MiniLM-L6-v2",
    "MPNet": "sentence-transformers/all-mpnet-base-v2",
    "DistilRoBERTa": "sentence-transformers/paraphrase-distilroberta-base-v1",
}

# 추가적인 모델:     "Instructor-xl": "hkunlp/instructor-xl",
#                   "E5-large-v2": "intfloat/e5-large-v2",
#                    OpenAI api

models = {name: SentenceTransformer(model_path) for name, model_path in model_names.items()}

# ===============================
# 2. 테스트용 식당 데이터
# ===============================
restaurants = [
    {
        "name": "Seoul Garden BBQ",
        "description": "A traditional Korean BBQ restaurant with premium cuts of meat. "
                       "Cozy, dimly lit interior with private booths, exuding a warm, familial vibe.",
        "menus": [
            "Marinated Beef Short Ribs",
            "Spicy Pork Belly",
            "Soy Garlic Chicken",
            "Kimchi Pancake",
            "Korean Rice Wine (Makgeolli)"
        ],
        "reviews": [
            "The meat quality is exceptional and the private booths create an intimate atmosphere.",
            "Every dish bursts with traditional flavors and the ambience feels like home."
        ]
    },
    {
        "name": "Bibimbap House",
        "description": "A casual Korean eatery focused on customizable bibimbap bowls. "
                       "Bright, modern space that radiates a clean and healthy aura.",
        "menus": [
            "Classic Bibimbap",
            "Spicy Pork Bibimbap",
            "Vegetarian Bibimbap",
            "Seaweed Soup",
            "Korean Plum Tea"
        ],
        "reviews": [
            "The ingredients are extremely fresh and the vibe is very light and modern.",
            "A perfect spot for a quick, nutritious meal without any pretension."
        ]
    },
    {
        "name": "Han River Soju Bar",
        "description": "A trendy Korean bar that serves traditional soju alongside fusion snacks. "
                       "Vibrant with neon accents and energetic music, yet maintaining an urban chic feel.",
        "menus": [
            "Original Soju",
            "Fruit-Infused Soju",
            "Spicy Fried Chicken",
            "Tteokbokki",
            "Kimchi Cheese Fries"
        ],
        "reviews": [
            "The soju selection is impressive and the atmosphere buzzes with youthful energy.",
            "Great for a night out – the food and drinks are on point and the vibe is lively."
        ]
    },
    {
        "name": "Grandma’s Kimchi Stew",
        "description": "A home-style Korean restaurant offering slow-cooked, authentic stews. "
                       "Rustic decor with wooden tables that evoke nostalgia and warmth.",
        "menus": [
            "Kimchi Stew",
            "Soybean Paste Stew",
            "Braised Short Ribs",
            "Korean Fried Mackerel",
            "Steamed Egg Custard"
        ],
        "reviews": [
            "This place delivers a taste of home with every spoonful of stew.",
            "Rich flavors and a truly comforting atmosphere reminiscent of grandma's kitchen."
        ]
    },
    {
        "name": "K-Town Street Eats",
        "description": "A vibrant Korean street food joint serving market-style dishes. "
                       "Bustling yet unpretentious, radiating the raw energy of traditional street vendors.",
        "menus": [
            "Hotteok (Korean Sweet Pancake)",
            "Spicy Rice Cakes",
            "Korean Corn Dog",
            "Egg Bread",
            "Bubble Tea"
        ],
        "reviews": [
            "The energy is infectious and every bite feels like a journey to the streets of Seoul.",
            "Casual, affordable, and bursting with authentic street flavors."
        ]
    },
    {
        "name": "Royal Korean Dining",
        "description": "A high-end Korean restaurant offering an elevated dining experience. "
                       "Elegant interiors with traditional elements and refined service that exude sophistication.",
        "menus": [
            "Korean Royal Court Cuisine Set",
            "Abalone Porridge",
            "Braised Short Ribs",
            "Jeju Black Pork BBQ",
            "Premium Ginseng Chicken Soup"
        ],
        "reviews": [
            "An unforgettable experience with meticulous presentation and delicate flavors.",
            "The service is impeccable and the ambience is as regal as the food."
        ]
    },
    {
        "name": "Midnight Ramyeon Bar",
        "description": "A late-night Korean noodle bar specializing in gourmet ramyeon. "
                       "Dimly lit and cozy, perfect for those quiet, introspective late nights.",
        "menus": [
            "Classic Shin Ramyeon",
            "Cheese Ramyeon",
            "Seafood Ramyeon",
            "Spicy Beef Ramyeon",
            "Korean Beer Pairing"
        ],
        "reviews": [
            "Ideal for a solitary late-night treat; the noodles are comforting and hearty.",
            "The subtle spiciness in the broth is a perfect remedy after a long day."
        ]
    },
    {
        "name": "K-BBQ & Jazz Lounge",
        "description": "A fusion of Korean BBQ and live jazz that creates a sophisticated yet relaxed vibe. "
                       "Chic interiors combined with smooth tunes deliver a unique dining experience.",
        "menus": [
            "Premium Wagyu Galbi",
            "Truffle Bulgogi",
            "Korean BBQ Platter",
            "Kimchi Cheese Fondue",
            "Handcrafted Cocktails"
        ],
        "reviews": [
            "The blend of fine dining with live jazz makes for a very classy and memorable evening.",
            "The attention to detail in both food and ambience is truly impressive."
        ]
    },
    {
        "name": "Seaside Korean Grill",
        "description": "A beachfront Korean BBQ restaurant with breathtaking ocean views. "
                       "Casual yet stylish, combining fresh seafood with classic BBQ in a serene setting.",
        "menus": [
            "Grilled Squid",
            "Fresh Oysters",
            "Soy Marinated Crab",
            "Spicy Seafood Soup",
            "Makgeolli Sangria"
        ],
        "reviews": [
            "The ocean view elevates the dining experience to a whole new level.",
            "Fresh seafood and an ambient setting make it perfect for a relaxed seaside dinner."
        ]
    },
    {
        "name": "Temple Korean Vegan Cuisine",
        "description": "A serene vegan restaurant inspired by Buddhist temple cuisine. "
                       "Minimalist design and a peaceful, contemplative atmosphere highlight its focus on organic ingredients.",
        "menus": [
            "Temple Bibimbap",
            "Lotus Root Pancakes",
            "Fermented Soybean Stew",
            "Wild Greens & Tofu Soup",
            "Traditional Korean Herbal Tea"
        ],
        "reviews": [
            "An oasis of calm with dishes that are both innovative and soul-soothing.",
            "The flavors are subtle and pure, perfectly complementing the meditative ambience."
        ]
    }
]

# ===============================
# 3. 임베딩 및 코사인 유사도 함수
# ===============================
def get_embedding(text: str, model: SentenceTransformer) -> np.ndarray:
    return model.encode(text)

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# ===============================
# 4. 추천 함수
# ===============================
def recommend_restaurant(menu_input: str, detail_input: str, model: SentenceTransformer):
    """
    Combine menu_input + detail_input into a single query,
    compute its embedding, and then for each restaurant, combine
    description + menus + reviews into one text to compute cosine similarity.
    Returns the best matching restaurant and its similarity score.
    """
    user_query = f"{menu_input} {detail_input}"
    user_query_emb = get_embedding(user_query, model)

    best_score = -1
    best_restaurant = None

    for r in restaurants:
        combined_text = " ".join([r["description"], " ".join(r["menus"]), " ".join(r["reviews"])])
        restaurant_emb = get_embedding(combined_text, model)
        score = cosine_similarity(user_query_emb, restaurant_emb)
        if score > best_score:
            best_score = score
            best_restaurant = r

    return best_restaurant, best_score

# ===============================
# 5. 테스트셋
# ===============================
test_queries = [
    # 프롬프트는 식당 설명의 단어와 겹치지 않으면서도 분위기를 미묘하게 표현
    ("Grilled Delicacy", "I desire an experience that feels introspective and soothing, "
                           "with understated elegance and a subtle sense of nostalgia."),
    ("Fusion Delight", "I seek a setting that is vibrant yet refined, exuding energy "
                         "without overt modern trappings."),
    ("Comforting Meal", "I crave a dining experience that is reminiscent of cherished family memories, "
                         "warm and quietly intimate."),
    ("Nighttime Noodles", "I wish for a late-night escape that is contemplative and mellow, "
                           "without any flashy elements."),
    ("Urban Feast", "I am looking for a venue that combines sophistication with a casual, unassuming charm, "
                    "evoking a serene urban retreat.")
]

# ===============================
# 6. 각 모델에 대해 테스트
# ===============================
for query_idx, (menu_input, detail_input) in enumerate(test_queries, start=1):
    print(f"--- Test Query {query_idx} ---")
    print(f"Menu: {menu_input}")
    print(f"Detail: {detail_input}")
    for model_name, model in models.items():
        recommended, score = recommend_restaurant(menu_input, detail_input, model)
        print(f"[Model: {model_name}] Recommended: {recommended['name']} (Score: {score:.4f})")
    print("\n")


--- Test Query 1 ---
Menu: Grilled Delicacy
Detail: I desire an experience that feels introspective and soothing, with understated elegance and a subtle sense of nostalgia.
[Model: MiniLM] Recommended: K-BBQ & Jazz Lounge (Score: 0.4336)
[Model: MPNet] Recommended: K-BBQ & Jazz Lounge (Score: 0.5531)
[Model: DistilRoBERTa] Recommended: Temple Korean Vegan Cuisine (Score: 0.4189)


--- Test Query 2 ---
Menu: Fusion Delight
Detail: I seek a setting that is vibrant yet refined, exuding energy without overt modern trappings.
[Model: MiniLM] Recommended: Temple Korean Vegan Cuisine (Score: 0.2713)
[Model: MPNet] Recommended: K-BBQ & Jazz Lounge (Score: 0.4771)
[Model: DistilRoBERTa] Recommended: Temple Korean Vegan Cuisine (Score: 0.3913)


--- Test Query 3 ---
Menu: Comforting Meal
Detail: I crave a dining experience that is reminiscent of cherished family memories, warm and quietly intimate.
[Model: MiniLM] Recommended: Temple Korean Vegan Cuisine (Score: 0.4567)
[Model: MPNet] Recommende